<a href="https://colab.research.google.com/github/sfwfr/gg/blob/master/22p21s0246_pontagorrn_W2HW1_27092020_%E0%B8%82%E0%B9%89%E0%B8%AD%E0%B8%A1%E0%B8%B9%E0%B8%A5%E0%B9%80%E0%B9%80%E0%B8%9A%E0%B8%9A_7_%E0%B8%A7%E0%B8%B1%E0%B8%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
!pip install catboost

     |████████████████████████████████| 66.1MB 53kB/s 


In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/sfwfr/sturdy-pancake69/master/p1gd")
df.head()

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
0,15-05-2020 00:00,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0
1,15-05-2020 00:00,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.0,6183645.0
2,15-05-2020 00:00,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.0,6987759.0
3,15-05-2020 00:00,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.0,7602960.0
4,15-05-2020 00:00,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.0,7158964.0


In [4]:
df['Time'] = df['DATE_TIME'].apply(lambda x : x.split(' ')[1])
df['Time']

0        00:00
1        00:00
2        00:00
3        00:00
4        00:00
         ...  
68773    23:45
68774    23:45
68775    23:45
68776    23:45
68777    23:45
Name: Time, Length: 68778, dtype: object

In [12]:
df['TOTAL_YIELD_1'] = df.groupby(['SOURCE_KEY','Time']).shift(4)['TOTAL_YIELD'] # 1 วันก่อน
df['TOTAL_YIELD_2'] = df.groupby(['SOURCE_KEY','Time']).shift(5)['TOTAL_YIELD'] # 2 วันก่อน
df['TOTAL_YIELD_3'] = df.groupby(['SOURCE_KEY','Time']).shift(6)['TOTAL_YIELD'] # 3 วันก่อน
df['TOTAL_YIELD_1'] = df.groupby(['SOURCE_KEY','Time']).shift(7)['TOTAL_YIELD'] # 4 วันก่อน
df['TOTAL_YIELD_2'] = df.groupby(['SOURCE_KEY','Time']).shift(8)['TOTAL_YIELD'] # 5 วันก่อน
df['TOTAL_YIELD_3'] = df.groupby(['SOURCE_KEY','Time']).shift(9)['TOTAL_YIELD'] # 6 วันก่อน
df['TOTAL_YIELD_3'] = df.groupby(['SOURCE_KEY','Time']).shift(10)['TOTAL_YIELD'] # 7 วันก่อน

In [13]:
 from sklearn.preprocessing  import LabelEncoder 
 df['SOURCE_KEY']= df[['SOURCE_KEY']].apply( LabelEncoder().fit_transform)['SOURCE_KEY']

In [14]:
df = df.drop(['DATE_TIME','PLANT_ID','Time','DAILY_YIELD','DC_POWER','AC_POWER'],axis =1)

In [15]:
tendf = [df[df.index % 10 == i] for i in range(10)] 
tendf

[       SOURCE_KEY  TOTAL_YIELD  TOTAL_YIELD_1  TOTAL_YIELD_2  TOTAL_YIELD_3
 0               0    6259559.0            NaN            NaN            NaN
 10             11    6316803.0            NaN            NaN            NaN
 20             21    7116151.0            NaN            NaN            NaN
 30             10    6271355.0            NaN            NaN            NaN
 40             20    6339380.0            NaN            NaN            NaN
 ...           ...          ...            ...            ...            ...
 68730          18    7028601.0      6982403.0      6975980.0      6959527.0
 68740           6    7273532.0      7227367.0      7220978.0      7204682.0
 68750          16    7261681.0      7215600.0      7209095.0      7192554.0
 68760           4    7408587.0      7361623.0      7355017.0      7338202.0
 68770          14    7415430.0      7369294.0      7362716.0      7346132.0
 
 [6878 rows x 5 columns],
        SOURCE_KEY  TOTAL_YIELD  TOTAL_YIELD_1  

In [16]:
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor

In [17]:
train = tendf[1:]
test  = tendf[0]

In [18]:
train = pd.concat(train)
train = train.dropna()


In [19]:
xgb = XGBRegressor()
xgb.fit(train.drop(['TOTAL_YIELD'],axis =1 ),train['TOTAL_YIELD'])
cb = CatBoostRegressor()
cb.fit(train.drop(['TOTAL_YIELD'],axis =1 ),train['TOTAL_YIELD'])
lbg = LGBMRegressor()
lbg.fit(train.drop(['TOTAL_YIELD'],axis =1 ),train['TOTAL_YIELD'])
rm = RandomForestRegressor(max_depth=6)
rm.fit( train.drop(['TOTAL_YIELD'],axis =1 ),train['TOTAL_YIELD'])

[09:44:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Learning rate set to 0.076201
0:	learn: 386921.8146796	total: 55.3ms	remaining: 55.2s
1:	learn: 358176.1725020	total: 62.4ms	remaining: 31.1s
2:	learn: 331600.0614558	total: 69.3ms	remaining: 23s
3:	learn: 306895.4000177	total: 76.2ms	remaining: 19s
4:	learn: 284234.7325530	total: 83.1ms	remaining: 16.5s
5:	learn: 263185.3622385	total: 89.8ms	remaining: 14.9s
6:	learn: 243614.8647079	total: 96.5ms	remaining: 13.7s
7:	learn: 225497.7206678	total: 103ms	remaining: 12.8s
8:	learn: 208793.1793826	total: 110ms	remaining: 12.1s
9:	learn: 193350.1754767	total: 117ms	remaining: 11.5s
10:	learn: 179249.0646212	total: 123ms	remaining: 11.1s
11:	learn: 166184.9079094	total: 130ms	remaining: 10.7s
12:	learn: 153795.4538580	total: 136ms	remaining: 10.3s
13:	learn: 142466.9528451	total: 143ms	remaining: 10.1s
14:	learn: 131942.4673382	total: 150ms	remaining: 9.83s
15:	learn:

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=6, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [20]:
test = test.dropna()

In [21]:
pred_1 = xgb.predict(test.drop(['TOTAL_YIELD'],  axis =1 ))
pred_2 = cb.predict(test.drop(['TOTAL_YIELD'],  axis =1 ))
pred_3  = lbg.predict(test.drop(['TOTAL_YIELD'],  axis =1 ))
pred_4  = rm.predict(test.drop(['TOTAL_YIELD'],  axis =1 ))

In [22]:
from sklearn.metrics import mean_absolute_error

In [23]:
print('xgb ' + str( mean_absolute_error(pred_1,test['TOTAL_YIELD'] )) )
print( 'catb ' + str( mean_absolute_error(pred_2,test['TOTAL_YIELD'] )))
print('lgb ' + str(mean_absolute_error(pred_3,test['TOTAL_YIELD'] ) ))
print('rm ' + str(mean_absolute_error(pred_4,test['TOTAL_YIELD'] ) ))
print('stack ' + str(mean_absolute_error(pred_2 * 0.2 + pred_3 * 0.7 + pred_1*0.1,test['TOTAL_YIELD'] )))

xgb 3800.8650239144126
catb 2833.4355411179663
lgb 2839.382928625069
rm 5930.526141665002
stack 2657.1591366365205



lgb *จะไม่ไกลเคียงสุดเเล้ว* มี catb ตีมาคู่กัน ผลดีขื่นกว่าเเบบ 3 วัน